In [9]:
import requests
import pandas as pd
import os

# Foursquare API token
fsq_api_token = os.environ.get('FSQ_API_KEY')  # Use os.environ.get to access the environment variable

foursqr_headers = {
    "accept": "application/json",
    "Authorization": fsq_api_token
}

# Load bike stations data from citybikedata.csv
bike_stations_df = pd.read_csv('citybikedata.csv')  # Use 'citybikedata.csv' instead of 'divvy_station_data.csv'

# List to store Foursquare data for each station
fsq_data = []

# Loop through each bike station
for index, row in bike_stations_df.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    
    # Build the Foursquare API request URL with a 1000m radius
    fsq_url = f"https://api.foursquare.com/v3/places/search?ll={latitude},{longitude}&radius=1000&categories=food,drinks&fields=name,location,categories,distance,rating"
    
    # Send a request to Foursquare
    fsq_response = requests.get(fsq_url, headers=foursqr_headers)
    
    # Parse the Foursquare response
    fsq_results = fsq_response.json().get("response", {}).get("venues", [])
    
    # List to store parsed Foursquare POI data
    all_fsq_pois = []
    
    for poi in fsq_results:
        categories = [category['name'] for category in poi.get('categories', [])]
        poi_data = {
            "POI Name": poi['name'],
            "Full POI Location": poi['location'],
            "Categories": categories,
            "Rating": poi.get('rating', None),
            "Coordinates": poi.get('location', {}).get('lat', None),
            "Distance from Station": poi.get('distance', None)
        }
        all_fsq_pois.append(poi_data)
    
    # Extract the "Distance from Station" values from all_fsq_pois
    distances = [poi["Distance from Station"] for poi in all_fsq_pois if poi["Distance from Station"] is not None]
    
    # Find the lowest distance value
    lowest_distance = min(distances) if distances else None
    
    # Create a dictionary with station and Foursquare data
    fsq_data.append({
        "Station Name": row['Station Name'],  # Update to match the column name in citybikedata.csv
        "Latitude": latitude,
        "Longitude": longitude,
        "POIs": all_fsq_pois,
        "Shortest FSQ Distance from Station": lowest_distance
    })

# Convert the Foursquare data to a DataFrame
df_fsq_data = pd.DataFrame(fsq_data)


# Save the DataFrame to a CSV file
df_fsq_data.to_csv('combined_foursquare_data.csv', index=False)


In [8]:
import requests
import pandas as pd
import os

# Yelp API token
yelp_api_token = os.environ.get('YELP_API_KEY')  # Use os.environ.get to access the environment variable

# Load bike stations data from citybikedata.csv
bike_stations_df = pd.read_csv('citybikedata.csv')  # Use 'citybikedata.csv' instead of 'divvy_station_data.csv'

# List to store combined Yelp data
combined_yelp_data = []

# Loop through each bike station
for index, row in bike_stations_df.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    
    # Yelp API request
    yelp_url = f"https://api.yelp.com/v3/businesses/search?latitude={latitude}&longitude={longitude}&categories=restaurant,bar,entertainment&sort_by=best_match&limit=20"
    yelp_headers = {
        "Authorization": f"Bearer {yelp_api_token}",
        "accept": "application/json"
    }
    yelp_response = requests.get(yelp_url, headers=yelp_headers)
    yelp_results = yelp_response.json().get("businesses", [])
    
    # List to store parsed Yelp POI data
    all_yelp_pois = []
    
    for poi in yelp_results:
        categories = [category['title'] for category in poi.get('categories', [])]
        poi_data = {
            "Review_count": poi.get('review_count', None),
            "POI Name": poi.get('name', None),
            "Full POI Location": poi.get('location', None),
            "Categories": categories,
            "Rating": poi.get('rating', None),
            "Coordinates": poi.get('coordinates', None),
            "Distance from Station": poi.get('distance', None),
            "Is_business_closed": poi.get('is_closed', None)
        }
        all_yelp_pois.append(poi_data)
    
    # Extract the "Distance from Station" values from all_yelp_pois
    distances = [poi["Distance from Station"] for poi in all_yelp_pois if poi["Distance from Station"] is not None]
    
    # Find the lowest distance value
    lowest_distance = min(distances) if distances else None
    
    # Create final Yelp data
    combined_yelp_data.append({
        "Station Name": row['Station Name'],  # Update to match the column name in citybikedata.csv
        "Latitude": latitude,
        "Longitude": longitude,
        "Available Bikes": row['Num_Bikes'],  # Update to match the column name in citybikedata.csv
        "Empty Slots": row['Empty_Slots'],  # Update to match the column name in citybikedata.csv
        'Yelp POIS data': all_yelp_pois,
        "Shortest Yelp Distance from Station": lowest_distance
    })

# Create DataFrame
combined_yelp_df = pd.DataFrame(combined_yelp_data)

# Save DataFrame to CSV
combined_yelp_df.to_csv('combined_yelp_data.csv', index=False)

print("Combined Yelp data saved to 'combined_yelp_data.csv'")


Combined Yelp data saved to 'combined_yelp_data.csv'
